# HADOOP ECOSYSTEM CONTINUED

First, let's start the daemons again:

First ssh:

In [ ]:
/usr/sbin/sshd

In [ ]:
netstat -tulpn | grep 22

And then hdfs:

In [ ]:
start-dfs.sh 2> /dev/null

And yarn:

In [ ]:
start-yarn.sh 2> /dev/null

And the job history server (so that PIG does not complain that the server does not respond):

In [ ]:
mr-jobhistory-daemon.sh --config $HADOOP_CONF_DIR start historyserver 2> /dev/null

Check whether hdfs and yarn daemons are running:

In [ ]:
jps

Check the status of hdfs:

In [ ]:
hdfs dfsadmin -report 2> /dev/null

And if hdfs is in safemode, stop safemode:

In [ ]:
hdfs dfsadmin -safemode leave 2> /dev/null

And start postgresql:

In [ ]:
/usr/lib/postgresql/11/bin/pg_ctl start -D /etc/postgresql/11/main/ -m smart

In [ ]:
psql -U postgres -d imdb2 -c "\dt+"

## HIVE

Hive is a data warehouse system and MapReduce wrapper that presents an SQL interface. So SQL programmers feel at home through Hive in the Hadoop Ecosystem, without writing MapReduce jobs explicitly

First start the metastore, the central repository to store metadata for hive tables:

You should rerun this if the hive or pig commands return an error saying that "thrift server does not respond"

In [ ]:
hive --service metastore &

And check whether the service is listening on its port:

In [ ]:
netstat -an | grep 9083

### IMPORT DATA WITH SQOOP

Sqoop can import the data as Hive tables with "--hive-import --create-hive-table" flags

First flush the directory for default import location:

In [ ]:
hdfs dfs -rm -r /user/jovyan/title_ratings/ 2> /dev/null

In [ ]:
sqoop import \
--connect jdbc:postgresql://localhost:5432/imdb2 \
--username postgres \
--table title_ratings \
--hive-import --create-hive-table --direct \
2> /dev/null

And view the imported file:

In [ ]:
hdfs dfs -find / -name "title*ratings" 2> /dev/null

In [ ]:
hdfs dfs -ls /user/hive/warehouse/ 2> /dev/null

And let's check from hive whether files are imported:

In [ ]:
hive -e 'show tables' 2> /dev/null

Let's try to read the file:

In [ ]:
hdfs dfs -cat /user/hive/warehouse/title_ratings/* 2> /dev/null | head

If we want to delete the table, we do it from the hive command, not by manually deleting the file:

In [ ]:
hive -e 'drop table title_ratings' 2> /dev/null

And cheeck again whether the table is delted:

In [ ]:
hive -e 'show tables' 2> /dev/null

In [ ]:
hdfs dfs -ls /user/hive/warehouse/ 2> /dev/null

Let's import title_ratings again for further commands:

In [ ]:
hdfs dfs -rm -r /user/jovyan/title_ratings/ 2> /dev/null

In [ ]:
sqoop import \
--connect jdbc:postgresql://localhost:5432/imdb2 \
--username postgres \
--table title_ratings \
--hive-import --create-hive-table --direct \
2> /dev/null

And check again:

In [ ]:
hive -e 'show tables' 2> /dev/null

### A SQOOP TO HIVE IMPORT EXERCISE

Now you'll import a postgresql database into hdfs as text files and hive tables

Now you'll 
- first import a publicly available sql dump into postgresql
- Then import the postgresql database into hdfs as text files and hive tables

The database is "World", containing list of cities, countries and languages. The link is

https://ftp.postgresql.org/pub/projects/pgFoundry/dbsamples/world/world-1.0/world-1.0.tar.gz

There are other sample databases following the links:

http://pgfoundry.org/projects/dbsamples

https://wiki.postgresql.org/wiki/Sample_Databases

First download the file

In [ ]:
wget https://ftp.postgresql.org/pub/projects/pgFoundry/dbsamples/world/world-1.0/world-1.0.tar.gz

And extract the archive:

In [ ]:
tar -xzvf world-1.0.tar.gz

Create a new database at postgresql:

In [ ]:
createdb -U postgres world2

And import the dump into the database:

In [ ]:
psql -U postgres world2 < dbsamples-0.1/world/world.sql

View the tables:

In [ ]:
psql -U postgres -d world2 -c "\dt"

If you've missed the steps, no worry. There is already a copy of the database named **world** in postgresql. We can query it

View the tables:

In [ ]:
psql -U postgres -d world -c "\dt"

And view the fields of the tables:

In [ ]:
psql -U postgres -d world -c "\d+ public.*"

And view some of the rows of the tables:

In [ ]:
psql -U postgres -d world -c "select * from city limit 10"

In [ ]:
psql -U postgres -d world -c "select * from country limit 10"

In [ ]:
psql -U postgres -d world -c "select * from countrylanguage limit 10"

We may also view its entity relationship diagram:

![aa](world_erd.png)

And let's import into hive:

First get the list of tables:

In [ ]:
tables=$(psql -U postgres -d world -t --pset="border=0" -c "\dt" | \
awk -F " " '{ print $2 }')

In [ ]:
echo "$tables"

Create a hive database to import into:

In [ ]:
hive -e "create database world" 2> /dev/null

And import each table in the postgresql database into hive database:

In [ ]:
echo "$tables" | while read l;
do
    sqoop import \
    --connect jdbc:postgresql://localhost:5432/world \
    --username postgres \
    --table $l \
    --hive-import \
    --create-hive-table \
    --hive-table world.$l \
    --direct \
    2> /dev/null
done

### HIVE OPERATIONS

First show tables:

In [ ]:
hive -e "show tables" 2> /dev/null

Show tables in world database:

In [ ]:
hive -e "show tables in world" 2> /dev/null

Then let's create a database from hive:

In [ ]:
hive -e "create database deneme" 2> /dev/null

List the databases:

In [ ]:
hive -e "show databases" 2> /dev/null

Delete deneme database:

In [ ]:
hive -e "drop database deneme" 2> /dev/null

And let's run a simple query inside a hive database:

In [ ]:
hive -e "select * from title_ratings limit 1;"  2> /dev/null

Now that we have a hive database on hdfs, similar to our postgresql database, we can run similar queries

In fact, under the hood, Hive converts the HiveQL query to a series of map reduce jobs

The plan of the conversion can be viewed by prefixing the statement with "explain"

In [ ]:
hive -e "explain
select * from title_ratings limit 1;" \
2> /dev/null

Hive is suitable for simpler queries. However as the queries get more complex and need a clearer definition of the dataflow, we should revert to a tool such as Pig

**Exercise 1:**

Please:
- Create an imdb database in hive
- Import the first 10.000 rows of title_episode table from imdb2 database on postgresql into imdb database in hive
- Check the the table is imported

Note that:
- You should use the **--query** argument with the sqoop (check Session 6 notebook)
- When you use **--query**, you cannot use **--table** argument at the same time
- When you use **--query**, you also have to specify a temporary import directory with the **--target-dir** option (it should be a non-existent directory)
- You should add "WHERE \\\\$CONDITIONS" to your query in sqoop

In [ ]:
pass1=
solution=$(cat ~/crypt/sqoop_to_hive.crypt | openssl enc -md sha256 -aes-128-cbc -a -d -salt -pass pass:$pass1 2> /dev/null)
echo "$solution"; echo
for l in "$solution"; do eval "${l}"; done

### A HIVE EXERCISE WITH WORLD DATABASE

See the tables in the world database:

In [ ]:
hive -e "show tables in world" 2> /dev/null

In order get information about tables, you can run:

In [ ]:
hive -e "show create table world.city;" 2> /dev/null

The output will return information on the schema (column names and types), file type and size information

You can repeat it for other tables in the database

Now let's run a very simple query:

In [ ]:
hive -e "use world;
select * from country limit 1;" 2> /dev/null

Now let's run a query to see the average lifeexpantancy of all countries:

In [ ]:
hive -e "use world;
select avg(lifeexpectancy) from country limit 10;" 2> /dev/null

Now the next task is:

- get the names (from country table) of the countries, official languages (from country languages) of which include english 

You should get:

```mysql
OK
American Samoa
Anguilla
Antigua and Barbuda
Australia
Barbados
Belize
Bermuda
United Kingdom
Virgin Islands, British
Cayman Islands
South Africa
Falkland Islands
Gibraltar
Guam
Hong Kong
Ireland
Christmas Island
Canada
Cocos (Keeling) Islands
Lesotho
Malta
Marshall Islands
Montserrat
Nauru
Niue
Norfolk Island
Palau
Northern Mariana Islands
Saint Helena
Saint Kitts and Nevis
Saint Lucia
Saint Vincent and the Grenadines
Samoa
Seychelles
Tokelau
Tonga
Turks and Caicos Islands
Tuvalu
New Zealand
Vanuatu
United States
Virgin Islands, U.S.
Zimbabwe
United States Minor Outlying Islands
Time taken: 11.424 seconds, Fetched: 44 row(s)
```

In [ ]:
hive -e "use world;
select c.name from country c left join
countrylanguage l on c.code=l.countrycode
where l.isofficial = true
and l.language = 'English';"  2> /dev/null

**Exercise 2:**

Query world database in hive such that:
- The query should report names of countries and its capital cities and also the proportion of the population of the capital to the country as capshare
- country table should be left joined to city table. Check the ER diagram above again
- filter for countries with population above 0
- order by descending capshare

You can first try your query on postgresql server's world database

The result should be:

```Bash
Singapore                              Singapore                          112.63619287917017
Gibraltar                              Gibraltar                          108.1
Macao                                  Macao                              92.49471458773785
Pitcairn                               Adamstown                          84.0
Saint Pierre and Miquelon              Saint-Pierre                       82.97142857142858
Falkland Islands                       Stanley                            81.8
Palau                                  Koror                              63.1578947368421
Djibouti                               Djibouti                           60.03134796238245
Cook Islands                           Avarua                             59.5
Qatar                                  Doha                               59.26544240400668
Western Sahara                         El-Aaiún                           57.67918088737201
Bahamas                                Nassau                             56.02605863192182
Seychelles                             Victoria                           53.246753246753244
Cayman Islands                         George Town                        51.578947368421055
Holy See (Vatican City State)          Città del Vaticano                 45.5
Svalbard and Jan Mayen                 Longyearbyen                       44.9375
Marshall Islands                       Dalap-Uliga-Darrit                 43.75
Norfolk Island                         Kingston                           40.0
Iceland                                Reykjavík                          39.13405017921147
Tuvalu                                 Funafuti                           38.333333333333336
Virgin Islands, British                Road Town                          38.095238095238095
Uruguay                                Montevideo                         37.03925681750075
New Caledonia                          Nouméa                             35.65093457943925
Armenia                                Yerevan                            35.47443181818182
Antigua and Barbuda                    Saint John´s                       35.294117647058826
Gabon                                  Libreville                         34.176182707993476
Niue                                   Alofi                              34.1
Faroe Islands                          Tórshavn                           33.818604651162794
Sao Tome and Principe                  São Tomé                           33.701360544217685
Lebanon                                Beirut                             33.516148689823275
Congo                                  Brazzaville                        32.27998640842677
Latvia                                 Riga                               31.529081758930783
Chile                                  Santiago de Chile                  30.92468608244034
Saint Kitts and Nevis                  Basseterre                         30.526315789473685
Libyan Arab Jamahiriya                 Tripoli                            30.008920606601247
Guyana                                 Georgetown                         29.500580720092916
Mongolia                               Ulan Bator                         29.064613072877535
Turks and Caicos Islands               Cockburn Town                      28.235294117647058
Aruba                                  Oranjestad                         28.188349514563107
Estonia                                Tallinn                            28.069830461367427
French Guiana                          Cayenne                            28.010497237569062
Christmas Island                       Flying Fish Cove                   28.0
Cocos (Keeling) Islands                West Island                        27.833333333333332
Andorra                                Andorra la Vella                   27.165384615384614
Liberia                                Monrovia                           26.94990488268865
Suriname                               Paramaribo                         26.85851318944844
Cyprus                                 Nicosia                            25.83808135683053
Peru                                   Lima                               25.19169589275972
Saint Helena                           Jamestown                          25.0
Mauritania                             Nouakchott                         24.99250936329588
Georgia                                Tbilisi                            24.863123993558776
Maldives                               Male                               24.825174825174827
Greenland                              Nuuk                               24.008928571428573
Bahrain                                al-Manama                          23.987034035656404
Martinique                             Fort-de-France                     23.810126582278482
Azerbaijan                             Baku                               23.116110680113785
Dominica                               Roseau                             22.877464788732393
Tonga                                  Nuku´alofa                         22.626262626262626
Cape Verde                             Praia                              22.149532710280372
Macedonia                              Skopje                             21.95153162055336
South Korea                            Seoul                              21.308212364443687
Cuba                                   La Habana                          20.141058834032677
Samoa                                  Apia                               19.944444444444443
Austria                                Wien                               19.87374873328555
Guinea-Bissau                          Bissau                             19.868095630667767
Jordan                                 Amman                              19.673421207948063
Hong Kong                              Victoria                           19.354718372161603
Dominican Republic                     Santo Domingo de Guzmán            18.951924661565627
Nicaragua                              Managua                            18.900275916436737
Réunion                                Saint-Denis                        18.80972818311874
Iraq                                   Baghdad                            18.758382003028338
Luxembourg                             Luxembourg [Luxemburg/Lëtzebuerg]  18.521918751434473
Montserrat                             Plymouth                           18.181818181818183
Hungary                                Budapest                           18.03759757846105
Vanuatu                                Port-Vila                          17.736842105263158
Sierra Leone                           Freetown                           17.511330861145446
Panama                                 Ciudad de Panamá                   16.504656862745097
Moldova                                Chisinau                           16.43607305936073
Belarus                                Minsk                              16.354044548651817
United Arab Emirates                   Abu Dhabi                          16.3332650553052
Croatia                                Zagreb                             15.800804828973844
Angola                                 Luanda                             15.701195837863022
Lithuania                              Vilnius                            15.627119102338787
Liechtenstein                          Vaduz                              15.613003095975232
Saudi Arabia                           Riyadh                             15.383903364650346
Tokelau                                Fakaofo                            15.0
Saint Vincent and the Grenadines       Kingstown                          15.0
Colombia                               Santafé de Bogotá                  14.793747784787694
Guinea                                 Conakry                            14.678465679676986
Central African Republic               Bangui                             14.495159059474412
Zambia                                 Lusaka                             14.36361653397317
Virgin Islands, U.S.                   Charlotte Amalie                   13.978494623655914
Lesotho                                Maseru                             13.794705062703205
Bulgaria                               Sofija                             13.701815429317902
Slovenia                               Ljubljana                          13.632457993761948
Botswana                               Gaborone                           13.132983970406904
Ireland                                Dublin                             12.764006251490027
Honduras                               Tegucigalpa                        12.550501156515034
Kyrgyzstan                             Bishkek                            12.54309427537774
Ecuador                                Quito                              12.44233749802309
United Kingdom                         London                             12.218357222164451
Turkmenistan                           Ashgabat                           12.123794572774164
Zimbabwe                               Harare                             12.083297626189047
Mauritius                              Port-Louis                         11.934369602763384
Taiwan                                 Taipei                             11.867864845434939
Northern Mariana Islands               Garapan                            11.794871794871796
Czech Republic                         Praha                              11.491676477169905
Palestine                              Gaza                               11.403805224121252
Norway                                 Oslo                               11.359294406609356
Yugoslavia                             Beograd                            11.31578947368421
Solomon Islands                        Honiara                            11.283783783783784
Puerto Rico                            San Juan                           11.22703540966658
Eritrea                                Asmara                             11.194805194805195
French Polynesia                       Papeete                            10.873617021276596
Haiti                                  Port-au-Prince                     10.757382631963026
Finland                                Helsinki [Helsingfors]             10.741476998046913
North Korea                            Pyongyang                          10.33320853612879
Thailand                               Bangkok                            10.293610645124513
Israel                                 Jerusalem                          10.193019141064822
Paraguay                               Asunción                           10.148762736535662
Iran                                   Teheran                            9.98322796963162
Egypt                                  Cairo                              9.91599094493939
Somalia                                Mogadishu                          9.87422006536595
Congo, The Democratic Republic of the  Kinshasa                           9.80369380880474
Namibia                                Windhoek                           9.791425260718425
Laos                                   Vientiane                          9.788330572427757
Fiji Islands                           Suva                               9.46952264381885
Denmark                                København                          9.300168855534709
Bolivia                                La Paz                             9.10242526113579
Cameroon                               Yaoundé                            9.100430891614186
Bosnia and Herzegovina                 Sarajevo                           9.06344410876133
Romania                                Bucuresti                          8.97833938233395
Equatorial Guinea                      Malabo                             8.830022075055188
Uzbekistan                             Toskent                            8.707541738629821
Mexico                                 Ciudad de México                   8.688533692013632
San Marino                             San Marino                         8.496296296296297
Tajikistan                             Dushanbe                           8.468002585649645
Sweden                                 Stockholm                          8.467601056266505
Costa Rica                             San José                           8.429803629132488
Syria                                  Damascus                           8.353488372093024
Slovakia                               Bratislava                         8.303702743253005
Senegal                                Dakar                              8.280466195548993
Venezuela                              Caracas                            8.172503103020274
Togo                                   Lomé                               8.101101749837978
Mayotte                                Mamoutzou                          8.053691275167786
Argentina                              Buenos Aires                       8.052889392957443
Albania                                Tirana                             7.9383746912854285
Afghanistan                            Kabul                              7.834507042253521
Kenya                                  Nairobi                            7.613031914893617
Wallis and Futuna                      Mata-Utu                           7.58
Anguilla                               The Valley                         7.4375
Myanmar                                Rangoon (Yangon)                   7.370371182390213
Greece                                 Athenai                            7.321202006505021
Spain                                  Madrid                             7.29951295202793
Guatemala                              Ciudad de Guatemala                7.231453667105841
Micronesia, Federated States of        Palikir                            7.226890756302521
Mali                                   Bamako                             7.206266690404131
Tunisia                                Tunis                              7.204256206968496
Chad                                   N´Djaména                          6.939811789308587
Burkina Faso                           Ouagadougou                        6.90290692803887
Algeria                                Alger                              6.888881827714404
El Salvador                            San Salvador                       6.6180050987890375
Brunei                                 Bandar Seri Begawan                6.55
Japan                                  Tokyo                              6.297828179995896
Comoros                                Moroni                             6.228373702422146
Swaziland                              Mbabane                            6.051587301587301
Malaysia                               Kuala Lumpur                       5.8331505124977525
Russian Federation                     Moscow                             5.709502225488995
Portugal                               Lisboa                             5.633452028486837
East Timor                             Dili                               5.4124293785310735
Ghana                                  Accra                              5.293884820898477
Ukraine                                Kyiv                               5.200570794355478
Mozambique                             Maputo                             5.177530487804878
Papua New Guinea                       Port Moresby                       5.138339920948616
Cambodia                               Phnom Penh                         5.1052560888252145
Grenada                                Saint George´s                     4.915957446808511
Nauru                                  Yaren                              4.658333333333333
Netherlands                            Amsterdam                          4.609178013111447
Italy                                  Roma                               4.583184812760056
Turkey                                 Ankara                             4.562416843116938
Indonesia                              Jakarta                            4.528327683669091
Burundi                                Bujumbura                          4.480955937266617
New Zealand                            Wellington                         4.316416364577939
Madagascar                             Antananarivo                       4.238295069627399
Poland                                 Warszawa                           4.179090692716849
Germany                                Berlin                             4.121802915363897
Uganda                                 Kampala                            4.0903664248324
Jamaica                                Kingston                           4.024854819976771
Malawi                                 Lilongwe                           3.990517162471396
Ethiopia                               Addis Abeba                        3.987852633261408
Niger                                  Niamey                             3.91425908667288
Rwanda                                 Kigali                             3.6984352773826457
Monaco                                 Monaco-Ville                       3.6294117647058823
France                                 Paris                              3.588384772151279
Sri Lanka                              Colombo                            3.4259308440006375
American Samoa                         Fagatogo                           3.416176470588235
Trinidad and Tobago                    Port-of-Spain                      3.351042471042471
Gambia                                 Banjul                             3.2433716475095786
Sudan                                  Khartum                            3.2128958969142083
Benin                                  Porto-Novo                         3.18189273413154
Belize                                 Belmopan                           2.9481327800829877
Bangladesh                             Dhaka                              2.7972978204482986
Yemen                                  Sanaa                              2.7804770318021204
Guadeloupe                             Basse-Terre                        2.726535087719298
Kiribati                               Bairiki                            2.6819277108433734
Nepal                                  Kathmandu                          2.473192645215211
Barbados                               Bridgetown                         2.248148148148148
Morocco                                Rabat                              2.1990652887023385
Philippines                            Manila                             2.0812747640422815
Oman                                   Masqat                             2.04441384736428
Kazakstan                              Astana                             1.9182641928126734
Malta                                  Valletta                           1.8603366649132036
Bermuda                                Hamilton                           1.8461538461538463
Vietnam                                Hanoi                              1.7662090389818619
Switzerland                            Bern                               1.7135914194737725
Australia                              Canberra                           1.7087948745102193
South Africa                           Pretoria                           1.6312009312232212
Saint Lucia                            Castries                           1.4941558441558442
Kuwait                                 Kuwait                             1.4634381338742393
Belgium                                Bruxelles [Brussel]                1.3073444672331282
Brazil                                 Brasília                           1.1579625547423802
Netherlands Antilles                   Willemstad                         1.0806451612903225
Canada                                 Ottawa                             1.0764343275435837
Bhutan                                 Thimphu                            1.0357815442561205
Côte dIvoire                           Yamoussoukro                       0.8792100635736507
Guam                                   Agaña                              0.6779761904761905
China                                  Peking                             0.5848658143113659
Tanzania                               Dodoma                             0.5638929498463466
Pakistan                               Islamabad                          0.3351801793166031
Nigeria                                Abuja                              0.313974135920937
United States                          Washington                         0.2055127049077264
India                                  New Delhi                          0.029723615958771266
```

In [ ]:
pass1=
solution=$(cat ~/crypt/hive_example.crypt | openssl enc -md sha256 -aes-128-cbc -a -d -salt -pass pass:$pass1 2> /dev/null)
echo "$solution"; echo
for l in "$solution"; do eval "${l}"; done